In [1]:
import os
os.chdir('../../')

In [2]:
import data
import preprocess_utils.session2vec as sess2vec
import utils.sparsedf as sparsedf
from utils.df import MinMaxScaler
from utils.dataset import SequenceDatasetForBinaryClassification
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from IPython.display import display
pd.options.display.max_columns = None
tqdm.pandas()

Using TensorFlow backend.


In [3]:
from recommenders.recurrent.RNNBinaryClassificator import RNNBinaryClassificator

from numpy.linalg import norm as L2Norm
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

import keras
from keras.models import load_model, Model
from keras.layers import Input, Dense, LSTM, GRU, Embedding, Dropout, TimeDistributed, concatenate
from keras.callbacks import EarlyStopping

In [4]:
mode = 'small'

## Prepare dataset

In [5]:
def add_reference_id(df):
    x_df = df.copy()
    embed_df = pd.read_csv('ref_embedding.csv').rename(columns={'index':'ref_id'})
    max_id = embed_df['ref_id'].max()+1
    x_df.loc[x_df.reference.str.isnumeric() != True, 'reference'] = 0
    x_df = x_df.astype({'reference':'int'})
    return x_df.merge(embed_df, how='left', left_on='reference', right_on='reference').fillna(max_id)

In [37]:
rows_per_sample = 12
path = f'dataset/preprocessed/cluster_recurrent/{mode}/dataset_classification_p{rows_per_sample}/'
x_df = pd.read_csv(path + '/X_train.csv', index_col=0).drop(['user_id','session_id','timestamp','step','platform','city','current_filters'], axis=1)
y_df = pd.read_csv(path + '/Y_train.csv', index_col=0).drop(['session_id','user_id','timestamp','step'], axis=1)
feat_x = len(x_df.columns)

In [38]:
x_df.head(12)

,reference,frequence,rp_0,rp_1,rp_2,rp_3,rp_4,rp_5,rp_6,rp_7,rp_8,rp_9,rp_10,rp_11,rp_12,rp_13,rp_14,rp_15,rp_16,rp_17,rp_18,rp_19,rp_20,rp_21,rp_22,rp_23,rp_24,glob_clickout_popularity,avg_price,price,pricepos_0,pricepos_1,pricepos_2,pricepos_3,pricepos_4,pricepos_5,pricepos_6,pricepos_7,pricepos_8,pricepos_9,pricepos_10,pricepos_11,pricepos_12,pricepos_13,pricepos_14,pricepos_15,pricepos_16,pricepos_17,pricepos_18,pricepos_19,pricepos_20,pricepos_21,pricepos_22,pricepos_23,pricepos_24,mobile,desktop,tablet,show_impression,clickout item,interaction item rating,interaction item info,interaction item image,interaction item deals,change of sort order,filter selection,search for item,search for destination,search for poi
orig_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
284016,"Orlando, USA",2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [41]:
sess2vec.merge_reference_features(x_df, rows_per_sample)

,reference,frequence,rp_0,rp_1,rp_2,rp_3,rp_4,rp_5,rp_6,rp_7,rp_8,rp_9,rp_10,rp_11,rp_12,rp_13,rp_14,rp_15,rp_16,rp_17,rp_18,rp_19,rp_20,rp_21,rp_22,rp_23,rp_24,glob_clickout_popularity,avg_price,price,pricepos_0,pricepos_1,pricepos_2,pricepos_3,pricepos_4,pricepos_5,pricepos_6,pricepos_7,pricepos_8,pricepos_9,pricepos_10,pricepos_11,pricepos_12,pricepos_13,pricepos_14,pricepos_15,pricepos_16,pricepos_17,pricepos_18,pricepos_19,pricepos_20,pricepos_21,pricepos_22,pricepos_23,pricepos_24,mobile,desktop,tablet,show_impression,clickout item,interaction item rating,interaction item info,interaction item image,interaction item deals,change of sort order,filter selection,search for item,search for destination,search for poi,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,Air Conditioning,Airport Hotel,Airport Shuttle,All Inclusive (Upon Inquiry),Balcony,Bathtub,Beach,Beach Bar,Beauty Salon,Bed & Breakfast,Bike Rental,Boat Rental,Body Treatments,Boutique Hotel,Bowling,Bungalows,Business Centre,Business Hotel,Cable TV,Camping Site,Car Park,Casa Rural (ES),Casino (Hotel),Central Heating,Childcare,Club Hotel,Computer with Internet,Concierge,Conference Rooms,Convenience Store,Convention Hotel,Cosmetic Mirror,Cot,Country Hotel,Deck Chairs,Design Hotel,Desk,Direct beach access,Diving,Doctor On-Site,Eco-Friendly hotel,Electric Kettle,Excellent Rating,Express Check-In / Check-Out,Family Friendly,Fan,Farmstay,Fitness,Flatscreen TV,Free WiFi (Combined),Free WiFi (Public Areas),Free WiFi (Rooms),Fridge,From 2 Stars,From 3 Stars,From 4 Stars,Gay-friendly,Golf Course,Good Rating,Guest House,Gym,Hairdresser,Hairdryer,Halal Food,Hammam,Health Retreat,Hiking Trail,Honeymoon,Horse Riding,Hostal (ES),Hostel,Hot Stone Massage,Hotel,Hotel Bar,House / Apartment,Hydrotherapy,Hypoallergenic Bedding,Hypoallergenic Rooms,Ironing Board,Jacuzzi (Hotel),Kids' Club,Kosher Food,Large Groups,Laundry Service,Lift,Luxury Hotel,Massage,Microwave,Minigolf,Motel,Nightclub,Non-Smoking Rooms,On-Site Boutique Shopping,Openable Windows,Organised Activities,Pet Friendly,Playground,Pool Table,Porter,Pousada (BR),Radio,Reception (24/7),Resort,Restaurant,Romantic,Room Service,Room Service (24/7),Safe (Hotel),Safe (Rooms),Sailing,Satellite TV,Satisfactory Rating,Sauna,Self Catering,Senior Travellers,Serviced Apartment,Shooting Sports,Shower,Singles,Sitting Area (Rooms),Ski Resort,Skiing,Solarium,Spa (Wellness Facility),Spa Hotel,Steam Room,Sun Umbrellas,Surfing,Swimming Pool (Bar),Swimming Pool (Combined Filter),Swimming Pool (Indoor),Swimming Pool (Outdoor),Szep Kartya,Table Tennis,Telephone,Teleprinter,Television,Tennis Court,Tennis Court (Indoor),Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms)
orig_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0

In [8]:
y_df.head()

,ref_0,ref_1,ref_2,ref_3,ref_4,ref_5,ref_6,ref_7,ref_8,ref_9,ref_10,ref_11,ref_12,ref_13,ref_14,ref_15,ref_16,ref_17,ref_18,ref_19,ref_20,ref_21,ref_22,ref_23,ref_24
orig_index,,,,,,,,,,,,,,,,,,,,,,,,,
284019,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
147484,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
205332,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
369577,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
446292,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
x_int, x_ref, y = x_df.drop('ref_id',axis=1).values.reshape((-1, rows_per_sample, feat_x-1)), x_df.ref_id.values.reshape((-1, rows_per_sample)), y_df.values
print(x_int.shape, x_ref.shape, y.shape)

(63247, 12, 68) (63247, 12) (63247, 25)


In [40]:
max_reference_id = x_df['ref_id'].max()
max_reference_id

331751

In [41]:
x_int, x_ref, y = shuffle(x_int, x_ref, y)

In [42]:
np.sum(y, axis=0)

array([23845,  6503,  4561,  3488,  3030,  2393,  2026,  1805,  1625,
        1433,  1355,  1262,  1163,  1006,   928,   868,   809,   706,
         733,   657,   643,   635,   567,   555,   651])

In [43]:
weights = np.sum(y, axis=0) / y.shape[0]
weights = 1/weights
wgt_sum = sum(weights)
weights = weights/wgt_sum
weights

array([0.00180806, 0.00662973, 0.00945257, 0.01236042, 0.01422877,
       0.01801636, 0.02127994, 0.02388541, 0.02653118, 0.03008595,
       0.03181783, 0.03416257, 0.03707064, 0.04285602, 0.04645815,
       0.04966954, 0.05329192, 0.0610668 , 0.05881741, 0.06562125,
       0.06705002, 0.06789474, 0.07603732, 0.07768137, 0.06622605])

## Build/load the model

In [44]:
interaction_input = Input((rows_per_sample, feat_x-1), name='interaction')
reference_input = Input((rows_per_sample, ), name='reference')

item_embedding = Embedding(max_reference_id+1, 64, name='reference_embedding', input_length=rows_per_sample, mask_zero=True)(reference_input)

dense_interaction = Dense(64)(interaction_input)

interaction_embed = concatenate([dense_interaction, item_embedding], name='interaction_embedding')

#m.add( TimeDistributed(Dense(64), input_shape=(6,68)) )
gru1 = GRU(64, recurrent_dropout=0.2, dropout=0.2, return_sequences=True)(interaction_embed)
gru2 = GRU(48, recurrent_dropout=0.2, dropout=0.2, return_sequences=False)(gru1)
dense1 = Dense(32, activation='relu')(gru2)
dropout1 = Dropout(0.2)(dense1)
dense2 = Dense(25, activation='sigmoid')(dropout1)
dropout2 = Dropout(0.1)(dense2)

m = Model(input=[interaction_input, reference_input], output=dropout2)

#sgd = keras.optimizers.SGD(lr=1e-6, decay=1e-6, momentum=0.9, nesterov=True)
#adam = keras.optimizers.Adam(lr=1e-3)
rmsprop = keras.optimizers.RMSprop()
m.compile(rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
interaction (InputLayer)        (None, 12, 68)       0                                            
__________________________________________________________________________________________________
reference (InputLayer)          (None, 12)           0                                            
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 12, 64)       4416        interaction[0][0]                
__________________________________________________________________________________________________
reference_embedding (Embedding) (None, 12, 64)       21232128    reference[0][0]                  
__________________________________________________________________________________________________
interactio

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`


## Train the model

In [ ]:
m.fit(x=[x_int, x_ref], y=y, epochs=100, validation_split=0.2, class_weight=weights, batch_size=32,
      callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 1444407424 elements. This may consume a large amount of memory.
  num_elements)


Train on 50597 samples, validate on 12650 samples
Epoch 1/100


In [23]:
m.save('gru_binary.h5')

## Load saved model

In [8]:
m = load_model('gru_binary.h5')

## Use the model

In [10]:
m.predict(dataset.load_Xtest()[0])

array([[0.06239783],
       [0.88753366],
       [0.62861437],
       ...,
       [0.62944716],
       [0.07085662],
       [0.63053685]], dtype=float32)

In [7]:
#X_test_df = pd.read_csv(f'dataset/preprocessed/cluster_recurrent/{mode}/X_test.csv').set_index('orig_index')
#X_test_df = scale_dataframe(X_test_df, ['impression_price'])

target_indices = data.target_indices(mode, 'cluster_recurrent')
#target_indices = data.target_indices(mode, 'cluster_recurrent_up_to_len6')
#X_test_df.head(3)
target_indices[:10]

array([17201, 17204, 17221, 17268, 17270, 17297, 17318, 17365, 17367,
       17459])

In [9]:
recomendations = model.recommend_batch(target_indices)
if mode != 'full':
    model.compute_MRR(recomendations)

caching df_full...


  3%|▎         | 94/3173 [00:00<00:03, 939.28it/s]

Done!


100%|██████████| 3173/3173 [00:01<00:00, 2755.35it/s]

prediction created !!!
Loading full train_df, it will take a while..



100%|██████████| 3173/3173 [00:00<00:00, 764945.77it/s]

Calculating MRR (hoping for a 0.99)
MRR: 0.6200074513933509


In [12]:
xtest, indices = dataset.load_Xtest()
y_true = sess2vec.add_reference_binary_labels(data.full_df().loc[indices], actiontype_col='action_type', action_equals='clickout item').ref_class

X_test: (15521, 6, 68)
caching df_full...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Done!


100%|██████████| 15521/15521 [00:26<00:00, 590.63it/s]


In [14]:
y_pred = m.predict_classes(xtest)
print(classification_report(y_true, y_pred, target_names=['1','0']))

              precision    recall  f1-score   support

           1       0.78      0.78      0.78      9625
           0       0.64      0.65      0.64      5896

   micro avg       0.73      0.73      0.73     15521
   macro avg       0.71      0.71      0.71     15521
weighted avg       0.73      0.73      0.73     15521



### Features: device, reference position, popularity, price, avg_price

#### 2x64 gru - 2 dense


In [ ]:
rec_df = pd.DataFrame(recomendations, columns=['id','recs'])
rec_df.head(10)

In [ ]:
data.train_df('full').loc[rec_df.id]

In [13]:
import out
out.create_sub(recomendations, submission_name='RNNclass_gru2x128_2dense_0617')

Exporting the sub to submissions/RNNclass_gru2x128_2dense_0617 18-38-09.csv...

submission created in submissions folder in 00m 03s


## Evaluate the model

In [8]:
def compute_precision(predictions):
    train_df = data.train_df('full')

    target_indices, recs = zip(*predictions)
    target_indices = list(target_indices)
    
    df_target = train_df.loc[target_indices][['reference','impressions']]
    correct_clickouts = df_target.reference.values.astype('int')
    impressions = df_target.impressions.str.split('|').apply(lambda x: list(map(int, x)) ).values
    classes = [ impressions[i].index(correct_clickouts[i]) for i in range(len(impressions)) ]
    
    len_rec = len(recs)
    
    true_positive = np.zeros((25))
    count_per_class = np.zeros((25))

    for i in tqdm(range(len_rec)):
        cur_class = classes[i]
        count_per_class[cur_class] += 1

        correct_clickout = correct_clickouts[i]
        if correct_clickout == recs[i][0]:
            true_positive[cur_class] += 1
    
    print(f'precision per class: {true_positive / count_per_class}')
    
    print(f'true positive: {true_positive}')
    print(f'counts: {count_per_class}')
    return MRR

In [9]:
def compute_MRR_and_wrong(predictions):
    train_df = data.train_df('full')

    target_indices, recs = zip(*predictions)
    target_indices = list(target_indices)
    
    df_target = train_df.loc[target_indices][['reference','impressions']]
    correct_clickouts = df_target.reference.values.astype('int')
    impressions = df_target.impressions.str.split('|').apply(lambda x: list(map(int, x)) ).values
    classes = [ impressions[i].index(correct_clickouts[i]) for i in range(len(impressions)) ]
    
    len_rec = len(recs)
    
    false_positive = np.zeros((25))
    count_per_class = np.zeros((25))

    RR = 0
    for i in tqdm(range(len_rec)):
        cur_class = classes[i]
        count_per_class[cur_class] += 1

        correct_clickout = correct_clickouts[i]
        if correct_clickout != recs[i][0]:
            wrong_class = impressions[i].index(recs[i][0])
            false_positive[wrong_class] += 1
        
        if correct_clickout in predictions[i][1]:
            rank_pos = recs[i].index(correct_clickout) + 1
            if rank_pos <= 25:
                RR += 1 / rank_pos

    MRR = RR / len_rec
    print(f'MRR: {MRR}')
    
    print(f'wrong per class: {false_positive / count_per_class}')
    
    print(f'false positive: {false_positive}')
    print(f'counts: {count_per_class}')
    return MRR

In [10]:
x_test = dataset.load_Xtest()
recomendations = model.recommend_batch(target_indices)

caching df_full...


  0%|          | 337/153132 [00:00<03:12, 794.01it/s]

Done!


100%|██████████| 153132/153132 [01:00<00:00, 2520.79it/s]

prediction created !!!


In [28]:
if mode != 'full':
    compute_MRR_and_precision(recomendations)


MRR: 0.6316681353381653
precision per class: [0.9645003  0.26290209 0.26725267 0.25985178 0.25618254 0.25621415
 0.25706732 0.26542226 0.26653154 0.26792556 0.25610501 0.25658363
 0.26413718 0.25807754 0.27155963 0.25847458 0.2723192  0.26372443
 0.26710291 0.26889169 0.23430421 0.26831037 0.28317956 0.26984127
 0.26534191]
true positive: [56023.  4111.  2858.  2209.  1875.  1474.  1264.  1166.  1052.   979.
   839.   721.   724.   639.   592.   549.   546.   490.   449.   427.
   362.   370.   399.   391.   454.]
counts: [58085. 15637. 10694.  8501.  7319.  5753.  4917.  4393.  3947.  3654.
  3276.  2810.  2741.  2476.  2180.  2124.  2005.  1858.  1681.  1588.
  1545.  1379.  1409.  1449.  1711.]


In [10]:
if mode != 'full':
    compute_MRR_and_wrong(recomendations)

Loading full train_df, it will take a while..



MRR: 0.6316681353381653
wrong per class: [1.06974262 0.13813391 0.10510567 0.10387013 0.10848477 0.10203372
 0.1171446  0.10266333 0.10412972 0.11001642 0.1037851  0.10533808
 0.09157242 0.0957189  0.09633028 0.08333333 0.08678304 0.0818084
 0.09696609 0.07556675 0.07831715 0.07614213 0.07877928 0.0731539
 0.04792519]
false positive: [62136.  2160.  1124.   883.   794.   587.   576.   451.   411.   402.
   340.   296.   251.   237.   210.   177.   174.   152.   163.   120.
   121.   105.   111.   106.    82.]
counts: [58085. 15637. 10694.  8501.  7319.  5753.  4917.  4393.  3947.  3654.
  3276.  2810.  2741.  2476.  2180.  2124.  2005.  1858.  1681.  1588.
  1545.  1379.  1409.  1449.  1711.]


In [5]:
aa = pd.read_csv('dataset/preprocessed/cluster_recurrent/small/dataset_binary_classification/Y_train.csv')
aa.ref_class.mean()

0.44609874288924783

In [6]:
aa.head()

,orig_index,user_id,session_id,timestamp,step,ref_class
0,284019,C2D58CUH9AOT,000263df674fa,1541450198,5,1
1,147484,5Q8OI15YEX61,00033ccbbcdd4,1541352840,18,0
2,205332,5F1EH91MDGA5,0005174c23dcb,1541426275,6,0
3,369577,92LF9L875RJ7,000543ecdcf0f,1541110960,38,0
4,446292,C8EN3LLTNQOB,0005c6da8ea8a,1541044299,2,0


In [27]:
aa.ref_class.mean()

0.44609874288924783